In this nb, I want to
1. create the Python context to run a hqq-qdora forward and backward, and
2. write simple kernels for those, and
3. call them from Python context

Note: I'm not using inheritance to produce cleaner code in this nb. The goal is to have a single class with all the functionality in one place. This will make it easier later to write kernels.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import tensor, cat, int32
from torch import float16 as fp16
from math import ceil

torch.set_printoptions(linewidth=200, precision=2, sci_mode=False)

In [2]:
def assert_close(a,b): assert torch.isclose(a,b,atol=1e-2).all()
def assert_somehow_close(a,b): assert torch.isclose(a,b,atol=0.07).all() # allow error of 0.07 due to quanting

___

## Implement qdora in Python

In [3]:
base_linear = nn.Linear(4,5, bias=False, dtype=fp16) # ignore bias for now
base_linear.weight.data

tensor([[-0.45,  0.12, -0.46,  0.42],
        [ 0.01,  0.23, -0.23, -0.09],
        [-0.41,  0.39,  0.25, -0.33],
        [-0.24,  0.09, -0.34,  0.23],
        [-0.46,  0.40,  0.15,  0.41]], dtype=torch.float16)

In [4]:
tst_x = torch.randn(4, dtype=fp16); tst_x

tensor([-0.47,  0.87, -1.61,  1.72], dtype=torch.float16)

In [5]:
tst_result = base_linear(tst_x); tst_result

tensor([ 1.79,  0.41, -0.44,  1.13,  1.03], dtype=torch.float16, grad_fn=<SqueezeBackward4>)

### Dummy Quanted Module

The quanted module looks like so:

In [6]:
class QuantedModule(nn.Module):
    def __init__(self, linear, group_size):
        super().__init__()
        self.data, self.zero_points, self.scales = self.quant(linear, group_size)
        self.in_features,self.out_features = linear.in_features,linear.out_features

    @staticmethod
    def quant(linear, group_size):
        # # Only done once, so okay to use unoptimized off-the-shelf code
        #
        # get model weights
        # quantize model weights; which gives 1. quantized data and 2. optional metadata (for hqq: zero point, scale per group)
        # store quantized data and metadata in new nn.Module
        # return it

        return linear.weight.data, 0., 1. # for now, let's not quant
    
    def dequant(self):
        # # Done many times, so should be part of an optimized kernel
        return (self.data-self.zero_points)*self.scales # todo: dequant per group

    def forward(self, x):
        return self.dequant()@x

In [7]:
quanted_lin = QuantedModule(base_linear, group_size=8)
quanted_lin

QuantedModule()

In [8]:
print(quanted_lin(tst_x))
assert_close(quanted_lin(tst_x), tst_result)

tensor([ 1.79,  0.42, -0.44,  1.13,  1.03], dtype=torch.float16)


### Dora Module

A dora module looks like so:

In [9]:
# check column-wise norm works as expected
t =  torch.arange(0,30, dtype=torch.float16).reshape(5,-1)
t_norm = t.norm(p=2,dim=1,keepdim=True)
t_magnitudes = ((t/t_norm)**2).sum(axis=1)
assert_close(t_magnitudes, torch.ones_like(t_magnitudes))

In [10]:
class DoraModule(nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.base = linear
        self.a = nn.Linear(linear.in_features, rank, bias=False, dtype=fp16)
        self.b = nn.Linear(rank, linear.out_features, bias=False, dtype=fp16)
        self.alpha = alpha
        self.m = nn.Parameter(linear.weight.norm(p=2, dim=1))
        # init a & b to 0 -- a should be inited differently, but for sake of simplicity, set it to 0 as well
        self.a.weight.data.zero_()
        self.b.weight.data.zero_()
        # Make base non-trainable
        for param in self.base.parameters(): param.requires_grad = False
        
    def forward(self, x):
        x = self.base(x) + self.b(self.a(x))
        col_norms =  (self.base.weight + self.b.weight @ self.a.weight).norm(p=2, dim=1).detach()
        x /= col_norms
        x *= self.m * self.alpha
        return x

In [11]:
dora = DoraModule(base_linear, rank=2, alpha=1); dora

DoraModule(
  (base): Linear(in_features=4, out_features=5, bias=False)
  (a): Linear(in_features=4, out_features=2, bias=False)
  (b): Linear(in_features=2, out_features=5, bias=False)
)

In [12]:
print(dora(tst_x))
assert_close(dora(tst_x), tst_result)

tensor([ 1.79,  0.41, -0.44,  1.13,  1.03], dtype=torch.float16, grad_fn=<MulBackward0>)


### Dummy QuantedDoraModule

Here's the QuantedDoraModule:

In [13]:
# combination of QuantedModule and DoraModule
class QuantedDoraModule(nn.Module):    
    def __init__(self, linear, group_size, rank, alpha):
        super().__init__()
        self.base = {k:v for k,v in zip(('data', 'zero_points', 'scales'), self.quant(linear, group_size))}        
        self.a = nn.Linear(linear.in_features, rank, bias=False, dtype=fp16)
        self.b = nn.Linear(rank, linear.out_features, bias=False, dtype=fp16)
        self.alpha = alpha
        self.m = nn.Parameter(linear.weight.norm(p=2, dim=1))
        # init a & b to 0 -- a should be inited differently, but for sake of simplicity, set it to 0 as well
        self.a.weight.data.zero_()
        self.b.weight.data.zero_()
    
    @staticmethod
    def quant(linear, group_size):
        # # Only done once, so okay to use unoptimized off-the-shelf code
        #
        # get model weights
        # quantize model weights; which gives 1. quantized data and 2. optional metadata (for hqq: zero point, scale per group)
        # store quantized data and metadata in new nn.Module
        # return it

        return linear.weight.data, 0., 1. # for now, let's not quant
    
    def dequant(self):
        # # Done many times, so should be part of an optimized kernel
        return (self.base['data']-self.base['zero_points'])*self.base['scales'] # todo: dequant per group
    
    def forward(self, x):
        x = self.dequant()@x + self.b(self.a(x))
        col_norms =  (self.dequant() + self.b.weight @ self.a.weight).norm(p=2, dim=1).detach()
        x /= col_norms
        x *= self.m * self.alpha
        return x

In [14]:
qdora = QuantedDoraModule(base_linear, group_size=8, rank=2, alpha=1); qdora

QuantedDoraModule(
  (a): Linear(in_features=4, out_features=2, bias=False)
  (b): Linear(in_features=2, out_features=5, bias=False)
)

In [15]:
print(qdora(tst_x))
assert_close(qdora(tst_x), tst_result)

tensor([ 1.79,  0.42, -0.44,  1.13,  1.03], dtype=torch.float16, grad_fn=<MulBackward0>)


### With actual 3-bit quanting

**Now let's actually quant.** We'll do symetric grouped quanting, as hqq does (todo: verify). Let's use 3bits.

Let's first do it by hand, for `group_size = 5`.

In [16]:
bits = 3

In [17]:
data = torch.randn((2,10), dtype=torch.float16); data
data

tensor([[-1.61, -0.52,  0.34,  0.17, -0.91, -0.62, -0.32, -0.69,  0.76,  0.81],
        [-0.62,  0.20, -0.22, -0.32,  1.87, -0.95,  1.05, -1.55, -1.51,  0.34]], dtype=torch.float16)

In [18]:
data_group = data.flatten()[:5]; data_group

tensor([-1.61, -0.52,  0.34,  0.17, -0.91], dtype=torch.float16)

In [19]:
min_,max_ = data_group.min(), data_group.max(); min_,max_

(tensor(-1.61, dtype=torch.float16), tensor(0.34, dtype=torch.float16))

In [20]:
data_group_normed = (data_group-min_)/(max_-min_) * (2**bits-1); data_group_normed

tensor([0.00, 3.91, 7.00, 6.38, 2.51], dtype=torch.float16)

In [21]:
data_group_normed.round()

tensor([0., 4., 7., 6., 3.], dtype=torch.float16)

In [22]:
def quantize_group(group, bits=bits):
    group -= group.min() # start at 0
    group /= group.max() # scale to [0,1]
    return (group * (2**bits-1)).round() # scale to [0, 2**bits-1]

quantize_group(data.flatten()[:5])

tensor([0., 4., 7., 6., 3.], dtype=torch.float16)

This can be done group-wise by reshaping into shape `(-1, group_size)` first.

In [23]:
data = torch.arange(0,30, dtype=torch.float16)

In [24]:
def quant(data, group_size, bits):
    shape = data.shape
    data = data.reshape(-1,group_size)

    min_, max_ = data.min(axis=-1, keepdim=True).values, data.max(axis=-1, keepdim=True).values

    zero = min_
    scale = (max_-min_) / (2**bits-1) 

    # note: can't use shorthand ops like -= as they modify tensor in-place
    data = data - zero # start at 0
    data = data / scale # scale to [0, 2**bits-1]
    data = data.round()

    return data.reshape(shape), scale, zero

In [25]:
qdata, qscale, qzero = quant(data, group_size=5, bits=3)

# reshape/flatten for better visibility
print(qdata.reshape(-1,5))
print(qscale.flatten())
print(qzero.flatten())

tensor([[0., 2., 4., 5., 7.],
        [0., 2., 4., 5., 7.],
        [0., 2., 4., 5., 7.],
        [0., 2., 4., 5., 7.],
        [0., 2., 4., 5., 7.],
        [0., 2., 4., 5., 7.]], dtype=torch.float16)
tensor([0.57, 0.57, 0.57, 0.57, 0.57, 0.57], dtype=torch.float16)
tensor([ 0.,  5., 10., 15., 20., 25.], dtype=torch.float16)


As expected:
- the 1st element in each group is now 0, as it was the min in that group
- the scales are identical
- the zeros are 5*i for i = 0,...,5

In [26]:
def dequant(qdata, scale, zero, group_size):
    shape = qdata.shape
    data = qdata.reshape(-1,group_size)
    data = data*scale + zero
    return data.reshape(shape)

In [27]:
dequant(qdata,qscale,qzero, group_size=5).reshape(-1,5)

tensor([[ 0.00,  1.14,  2.29,  2.86,  4.00],
        [ 5.00,  6.14,  7.29,  7.86,  9.00],
        [10.00, 11.14, 12.28, 12.86, 14.00],
        [15.00, 16.14, 17.28, 17.86, 19.00],
        [20.00, 21.14, 22.28, 22.86, 24.00],
        [25.00, 26.14, 27.28, 27.86, 29.00]], dtype=torch.float16)

In [28]:
data.reshape(-1,5)

tensor([[ 0.,  1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.],
        [20., 21., 22., 23., 24.],
        [25., 26., 27., 28., 29.]], dtype=torch.float16)

Looks good! Let's implement this into the new version of `QuantedDoraModule`.

In [29]:
class QuantedDoraModule(nn.Module):
    def __init__(self, linear, bits, group_size, rank, alpha):
        super().__init__()
        # for quanting
        self.bits,self.group_size, = bits,group_size
        self.quant(linear)
        # for dora
        self.a = nn.Linear(linear.in_features, rank, bias=False, dtype=fp16)
        self.b = nn.Linear(rank, linear.out_features, bias=False, dtype=fp16)
        self.alpha = alpha
        self.m = nn.Parameter(linear.weight.norm(p=2, dim=1))
        # init a & b to 0 -- a should be inited differently, but for sake of simplicity, set it to 0 as well
        self.a.weight.data.zero_()
        self.b.weight.data.zero_()

    def quant(self, linear):
        data = linear.weight.data
        shape = data.shape

        # repeat last element, to have a multiple of group_size elements
        # note: element to pad with mustn't change any attribute that's use for quanting (eg min & max in a group)
        n_pad = data.numel()%self.group_size
        data = F.pad(data, (0,n_pad), 'constant', data.flatten()[-1])
        assert data.numel()%self.group_size==0

        data = data.reshape(-1,self.group_size)
        
        min_, max_ = data.min(axis=-1, keepdim=True).values, data.max(axis=-1, keepdim=True).values
        
        self.zero = min_
        self.scale = (max_-min_) / (2**self.bits-1) 
        
        # note: can't use shorthand ops like -= as they modify tensor in-place
        data = data - self.zero # start at 0
        data = data / self.scale # scale to [0, 2**bits-1]
        data = data.round()
        
        self.qdata = data.reshape(shape)

    def dequant(self):
        # # Done many times, so should be part of an optimized kernel
        shape = self.qdata.shape
        data = self.qdata.reshape(-1,self.group_size)
        data = data*self.scale + self.zero
        return data.reshape(shape)
        
    def forward(self, x):
        x = self.dequant()@x + self.b(self.a(x))
        col_norms =  (self.dequant() + self.b.weight @ self.a.weight).norm(p=2, dim=1).detach()
        x /= col_norms
        x *= self.m * self.alpha
        return x

In [30]:
qdora = QuantedDoraModule(base_linear, bits=3, group_size=5, rank=2, alpha=1); qdora

QuantedDoraModule(
  (a): Linear(in_features=4, out_features=2, bias=False)
  (b): Linear(in_features=2, out_features=5, bias=False)
)

In [31]:
print(f'quanted result: {qdora(tst_x)}')
print(f'exact   result: {tst_result}')
assert_somehow_close(qdora(tst_x), tst_result)

quanted result: tensor([ 1.80,  0.40, -0.45,  1.14,  1.00], dtype=torch.float16, grad_fn=<MulBackward0>)
exact   result: tensor([ 1.79,  0.41, -0.44,  1.13,  1.03], dtype=torch.float16, grad_fn=<SqueezeBackward4>)


In [32]:
tst_result_3bit_quanting = qdora(tst_x) # save to compare against later

### With Packing

**Now, let's store the quanted values more efficiently**, by storing 10 quanted 3bit-values together in a uint32 ('packing').

In [33]:
data = torch.arange(25, dtype=fp16)
n_data = data.numel()
n_packs = tensor(n_data/10).ceil().to(dtype=int)

print(f'We want to pack this data:\n{data}\nwhich has {n_data} items into {n_packs} uint32s.')

We want to pack this data:
tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.], dtype=torch.float16)
which has 25 items into 3 uint32s.


In [34]:
qdata, scales, zeros = quant(data, group_size=5, bits=3)
qdata, scales, zeros

(tensor([0., 2., 4., 5., 7., 0., 2., 4., 5., 7., 0., 2., 4., 5., 7., 0., 2., 4., 5., 7., 0., 2., 4., 5., 7.], dtype=torch.float16),
 tensor([[0.57],
         [0.57],
         [0.57],
         [0.57],
         [0.57]], dtype=torch.float16),
 tensor([[ 0.],
         [ 5.],
         [10.],
         [15.],
         [20.]], dtype=torch.float16))

In [35]:
to_pack_0 = qdata[:10]; to_pack_0

tensor([0., 2., 4., 5., 7., 0., 2., 4., 5., 7.], dtype=torch.float16)

In [36]:
# pack
pack_0 = 0
for x in to_pack_0: pack_0 = (pack_0 << 3) | int(x) # shift right 3 bits, then set last 3 bits to x
pack_0

43484463

In [37]:
# unpack
binary = [int(b) for b in format(pack_0, '032b')]
print(binary)

[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1]


In [38]:
def bin_to_dec(b3,b2,b1): return 4*b3 + 2*b2 + b1
assert bin_to_dec(1,1,1)==7
assert bin_to_dec(1,0,1)==5

In [39]:
[
    bin_to_dec(*binary[2+3*i:2+3*(i+1)]) # first 2 bits are not used; then every 3 bits represent one value
    for i in range(10)
]


[0, 2, 4, 5, 7, 0, 2, 4, 5, 7]

Yup, these are the values we originally packed.

The unpacking method uses an intermediate string representation. That is slow. Let's unpack on the raw binary.

In [40]:
[
    (pack_0 >> (3*i)) & 0b111 # righ-shift 3*i times, so last 3 bits are those we want; then only select those via 0b111
    for i in reversed(range(10))
]

[0, 2, 4, 5, 7, 0, 2, 4, 5, 7]

Yes!

Let's createa a packing and unpacking function now.

In [41]:
import math

In [42]:
# pack 10 3bit values into a 32bit val
def pack(vals):
    for v in vals: assert 0<=v<=7 and v//1==v, f'Value {v} can\'t be represented by 3 bits or is not an integer'
    
    n_packs = math.ceil(len(vals)/10)

    # pad with 0, to have a multiple of pack_size elements
    n_pad = n_packs*10 - len(vals)
    vals = F.pad(vals, (0,n_pad), 'constant', 0)
    assert len(vals)==n_packs*10

    packed = torch.zeros(n_packs, dtype=torch.int32)
    for i in range(n_packs):
        # pack the 10 vals from 10*i to 10*(i+1) into packed[i]
        for x in vals[10*i:10*(i+1)]: packed[i] = (packed[i] << 3) | x # shift right 3 bits, then set last 3 bits to x
    return packed

In [43]:
tst_values = tensor([0,1,2,3,4,5,6,7,0,1,2,3,4,5])

packed = pack(tst_values); packed

tensor([ 21913025, 328466432], dtype=torch.int32)

In [44]:
# unpack a 32bit value into 10 3bit vals
def unpack(packed):
    def bin_to_dec(b3,b2,b1): return 4*b3 + 2*b2 + b1
    for v in packed: isinstance(v, int), f'Value {v} is not an integer'
    unpacked = []
    for pack in packed:
        for i in reversed(range(10)):
            unpacked.append((pack >> (3*i)) & 0b111) # righ-shift 3*i times, so last 3 bits are those we want; then only select those via 0b111            
    return tensor(unpacked)

In [45]:
unpack(packed)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 0], dtype=torch.int32)

In [46]:
print(unpack(packed))
print(tst_values)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
tensor([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5])


Look's good!

Time to put this into `QuantedDoraModule`:

In [47]:
class QuantedDoraModule(nn.Module):
    def __init__(self, linear, bits, group_size, rank, alpha):
        super().__init__()
        # for quanting
        assert base_linear.weight.numel() % group_size ==0, f'group_size {group_size} can\'t cleanly split weight of base layer ({base_linear.weight.numel()} items)'
        self.bits,self.group_size, = bits,group_size
        self.quant(linear)
        # for dora
        self.a = nn.Linear(linear.in_features, rank, bias=False, dtype=fp16)
        self.b = nn.Linear(rank, linear.out_features, bias=False, dtype=fp16)
        self.alpha = alpha
        self.m = nn.Parameter(linear.weight.norm(p=2, dim=1))
        # init a & b to 0 -- a should be inited differently, but for sake of simplicity, set it to 0 as well
        self.a.weight.data.zero_()
        self.b.weight.data.zero_()

    def quant(self, linear):
        data = linear.weight.data
        self.shape = data.shape

        # repeat last element, to have a multiple of group_size elements
        # note: element to pad with mustn't change any attribute that's use for quanting (eg min & max in a group)
        n_pad = data.numel()%self.group_size
        data = F.pad(data, (0,n_pad), 'constant', data.flatten()[-1])
        assert data.numel()%self.group_size==0

        data = data.reshape(-1,self.group_size)
        
        min_, max_ = data.min(axis=-1, keepdim=True).values, data.max(axis=-1, keepdim=True).values
        
        self.zero = min_
        self.scale = (max_-min_) / (2**self.bits-1) 
        
        # note: can't use shorthand ops like -= as they modify tensor in-place
        data = data - self.zero # start at 0
        data = data / self.scale # scale to [0, 2**bits-1]
        data = data.round().to(int)

        # packed quantized data
        self.pqdata = self.pack(data.flatten())

    # pack 10 3bit values into a 32bit val
    @staticmethod
    def pack(vals):
        for v in vals: assert 0<=v<=7 and v//1==v, f'Value {v} can\'t be represented by 3 bits or is not an integer'
        
        n_packs = math.ceil(len(vals)/10)
    
        # pad with 0, to have a multiple of pack_size elements
        n_pad = n_packs*10 - len(vals)
        vals = F.pad(vals, (0,n_pad), 'constant', 0)
        assert len(vals)==n_packs*10
    
        packed = torch.zeros(n_packs, dtype=int32)
        for i in range(n_packs):
            # pack the 10 vals from 10*i to 10*(i+1) into packed[i]
            for x in vals[10*i:10*(i+1)]: packed[i] = (packed[i] << 3) | x # shift right 3 bits, then set last 3 bits to x
        return packed

    def dequant(self):
        data = self.unpack(self.pqdata)[:self.shape.numel()] # unpack & remove padding that was added during packing
        data = data.reshape(-1,self.group_size)
        data = data*self.scale + self.zero
        return data.reshape(self.shape)
    
    # unpack a 32bit value into 10 3bit vals
    @staticmethod
    def unpack(packed):
        def bin_to_dec(b3,b2,b1): return 4*b3 + 2*b2 + b1
        for v in packed: isinstance(v, int), f'Value {v} is not an integer'
        unpacked = []
        for pack in packed:
            for i in reversed(range(10)):
                unpacked.append((pack >> (3*i)) & 0b111) # righ-shift 3*i times, so last 3 bits are those we want; then only select those via 0b111            
        return tensor(unpacked)
    
    def forward(self, x):
        x = self.dequant()@x + self.b(self.a(x))
        col_norms =  (self.dequant() + self.b.weight @ self.a.weight).norm(p=2, dim=1).detach()
        x /= col_norms
        x *= self.m * self.alpha
        return x

In [48]:
qdora = QuantedDoraModule(base_linear, bits=3, group_size=5, rank=2, alpha=1); qdora

QuantedDoraModule(
  (a): Linear(in_features=4, out_features=2, bias=False)
  (b): Linear(in_features=2, out_features=5, bias=False)
)

In [49]:
print(f'quanted result (with packing): {qdora(tst_x)}')
print(f'quanted result (w/o  packing): {tst_result_3bit_quanting}')
print(f'exact   result               : {tst_result}')
assert_somehow_close(qdora(tst_x), tst_result)

quanted result (with packing): tensor([ 1.80,  0.40, -0.45,  1.14,  1.00], dtype=torch.float16, grad_fn=<MulBackward0>)
quanted result (w/o  packing): tensor([ 1.80,  0.40, -0.45,  1.14,  1.00], dtype=torch.float16, grad_fn=<MulBackward0>)
exact   result               : tensor([ 1.79,  0.41, -0.44,  1.13,  1.03], dtype=torch.float16, grad_fn=<SqueezeBackward4>)


Packing doesn't change the result -- as it should! Very good.

Let's call backwards on the model

In [61]:
# assert only the dora part is trainable
assert {n for n,p in qdora.named_parameters()} == {'m','a.weight','b.weight'}

In [57]:
y_is = (tst_result*2).detach()
y_pred = qdora(tst_x)

print(f'Result is    : {y_pred}')
print(f'Result should: {y_is}')

Result is    : tensor([ 1.80,  0.40, -0.45,  1.14,  1.00], dtype=torch.float16, grad_fn=<MulBackward0>)
Result should: tensor([ 3.58,  0.83, -0.88,  2.26,  2.05], dtype=torch.float16)


In [63]:
loss = (y_pred-y_is).square().sum().sqrt()
loss

tensor(2.42, dtype=torch.float16, grad_fn=<SqrtBackward0>)

In [64]:
loss.backward()

In [73]:
for n,p in qdora.named_parameters():
    print(f'Shape of loss of {n:<8} is {str(list(p.grad.shape)):<7}; shape of  {n:<8} is {list(p.shape)}')

Shape of loss of m        is [5]    ; shape of  m        is [5]
Shape of loss of a.weight is [2, 4] ; shape of  a.weight is [2, 4]
Shape of loss of b.weight is [5, 2] ; shape of  b.weight is [5, 2]


**Q:** How can we tell PyTorch to use a custom kernel for a sequence of operations? Especially for backwards, which is an op we don't define ourself.

## Kernels

In [ ]:
import os
os.environ['TRITON_INTERPRET'] = '1'

In [ ]:
# todo

___